In [1]:
import pandas as pd

dataSet = [['A','C','D'],
          ['B','C','E'],
          ['A','B','C','E'],
          ['B','E']]


In [2]:
import itertools

def createC1(dataSet):
    C1 = set(itertools.chain(*dataSet))
    
    return [frozenset([i]) for i in C1]

In [3]:
C1 = createC1(dataSet)
list(C1)[:5]

[frozenset({'A'}),
 frozenset({'B'}),
 frozenset({'D'}),
 frozenset({'C'}),
 frozenset({'E'})]

In [4]:
def scanD(dataSet, Ck, min_support):
    support = {}
    
    for i in dataSet:
        for j in Ck:
            if j.issubset(i):
                support[j] = support.get(j, 0) + 1
                
    n = len(dataSet)
    return{k: v/n for k, v in support.items() if v/n >= min_support}


In [5]:
min_support = 0.4
L1 = scanD(dataSet, C1, min_support)
L1

{frozenset({'A'}): 0.5,
 frozenset({'C'}): 0.75,
 frozenset({'B'}): 0.75,
 frozenset({'E'}): 0.75}

In [10]:
def aprioriGen(Lk):
    lenLk = len(Lk)
    k = len(Lk[0])
    print(lenLk, k)
    if lenLk > 1 and k > 0:
        return set([Lk[i].union(Lk[j])
                    for i in range(lenLk - 1)
                    for j in range(i + 1, lenLk)
                    if len(Lk[i] | Lk[j]) == k+1
                ])

In [12]:
C2 = aprioriGen(list(L1.keys()))
C2

4 1


{frozenset({'B', 'E'}),
 frozenset({'A', 'B'}),
 frozenset({'B', 'C'}),
 frozenset({'A', 'E'}),
 frozenset({'C', 'E'}),
 frozenset({'A', 'C'})}

In [8]:
min_support = 0.4
L2 = scanD(dataSet, C2, min_support)
L2

{frozenset({'A', 'C'}): 0.5,
 frozenset({'B', 'E'}): 0.75,
 frozenset({'B', 'C'}): 0.5,
 frozenset({'C', 'E'}): 0.5}

In [9]:
def apriori(dataSet, min_support=0.5):
    C1 = createC1(dataSet)
    L1 = scanD(dataSet, C1, min_support)
    L = [L1, ]
    k = 2
    while len(L[k-2]) > 1:
        Ck = aprioriGen(list(L[k-2].keys()))
        Lk = scanD(dataSet, Ck, min_support)
        if len(Lk) > 0:
            L.append(Lk)
            k += 1
        else:
            break
    d = {}
    for Lk in L:
        d.update(Lk)
    return d

In [10]:
L = apriori(dataSet, min_support)
L

4 1
4 2


{frozenset({'A'}): 0.5,
 frozenset({'C'}): 0.75,
 frozenset({'E'}): 0.75,
 frozenset({'B'}): 0.75,
 frozenset({'A', 'C'}): 0.5,
 frozenset({'B', 'E'}): 0.75,
 frozenset({'B', 'C'}): 0.5,
 frozenset({'C', 'E'}): 0.5,
 frozenset({'B', 'C', 'E'}): 0.5}

In [11]:
def rulesGen(iterable):
    subSet = []
    for i in range(1, len(iterable)):
        subSet.extend(itertools.combinations(iterable, i))
        
    return [(frozenset(lhs), frozenset(iterable.difference(lhs))) for lhs in subSet]

In [12]:
def arules(dataSet, min_support=0.5):
    L = apriori(dataSet, min_support)

    rules = []
    for Lk in L.keys():
        if len(Lk) > 1:
            rules.extend(rulesGen(Lk))
    
    scl = []
    for rule in rules:
        lhs = rule[0]; rhs = rule[1]
        support = L[lhs | rhs]
        confidence = support / L[lhs]
        lift = confidence / L[rhs]
        scl.append({'LHS':lhs, 'RHS':rhs, 'support':support, 'confidence':confidence, 'lift':lift})
    
    return pd.DataFrame(scl)

In [14]:
res = arules(dataSet, 0.4)
res

4 1
4 2


,LHS,RHS,support,confidence,lift
0,(A),(C),0.50,1.000000,1.333333
1,(C),(A),0.50,0.666667,1.333333
2,(B),(E),0.75,1.000000,1.333333
3,(E),(B),0.75,1.000000,1.333333
4,(B),(C),0.50,0.666667,0.888889
5,(C),(B),0.50,0.666667,0.888889
6,(E),(C),0.50,0.666667,0.888889
7,(C),(E),0.50,0.666667,0.888889
8,(B),"(E, C)",0.50,0.666667,1.333333
9,(E),"(B, C)",0.50,0.666667,1.333333


In [30]:
rules = Apriori.arules(dataSet, min_support)

AttributeError: 'DataFrame' object has no attribute 'reindex_axis'